<a href="https://colab.research.google.com/github/teamvictory1/Projects/blob/master/YSO_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Work on the project of customer analytic for YSO challenge


This notebook will be regularly updated throughout the duration of the project. In order to run this notebook, click on the Google Colab link.

The data is extracted from the Job Recommendation Challenge on Kaggle. To use this python notebook and kaggle dataset, it is essential to register for Kaggle API. To learn more about Kaggle API, read [here](https://github.com/Kaggle/kaggle-api).It is essential to agree to the T&C of the competition first before using the data. To use the Kaggle API, download the Kaggle json file

 For people who are not familiar with Google Colab, please read [here](https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d) for more information.

*Install PySpark*

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.mirror.digitalpacific.com.au/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!sudo tar -xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

**Testing the data from Kaggle challenge for job recommendation**

*Data Collection*

In [4]:
# Upload kaggle.json
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kienvu145","key":"701362c4f8962b18cb4419a33a068f3d"}'}

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [7]:
! kaggle competitions download -c job-recommendation

 88% 63.0M/71.6M [00:00<00:00, 72.3MB/s]
100% 71.6M/71.6M [00:00<00:00, 112MB/s] 
  0% 0.00/229k [00:00<?, ?B/s]
100% 229k/229k [00:00<00:00, 199MB/s]
 96% 66.0M/69.1M [00:00<00:00, 66.0MB/s]
100% 69.1M/69.1M [00:00<00:00, 101MB/s] 
 98% 33.0M/33.6M [00:01<00:00, 30.3MB/s]
100% 33.6M/33.6M [00:01<00:00, 27.8MB/s]
 61% 14.0M/23.0M [00:00<00:00, 35.0MB/s]
100% 23.0M/23.0M [00:00<00:00, 66.0MB/s]
  0% 0.00/2.66k [00:00<?, ?B/s]
100% 2.66k/2.66k [00:00<00:00, 2.92MB/s]
 29% 5.00M/17.0M [00:00<00:00, 21.3MB/s]
100% 17.0M/17.0M [00:00<00:00, 48.9MB/s]
  0% 0.00/76.2k [00:00<?, ?B/s]
100% 76.2k/76.2k [00:00<00:00, 72.1MB/s]
 30% 5.00M/16.5M [00:00<00:00, 14.4MB/s]
100% 16.5M/16.5M [00:00<00:00, 40.9MB/s]
 62% 5.00M/8.11M [00:00<00:00, 20.3MB/s]
100% 8.11M/8.11M [00:00<00:00, 27.0MB/s]
 99% 665M/671M [00:08<00:00, 92.1MB/s]
100% 671M/671M [00:08<00:00, 82.3MB/s]
 99% 662M/671M [00:08<00:00, 80.3MB/s]
100% 671M/671M [00:08<00:00, 80.3MB/s]
  0% 0.00/493 [00:00<?, ?B/s]
100% 493/493 [00:00<00:00

**EDA**

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

*App.tsv*

In [0]:
app = pd.read_csv('apps.tsv',delimiter='\t',encoding='utf-8')

In [10]:
app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1603111 entries, 0 to 1603110
Data columns (total 5 columns):
UserID             1603111 non-null int64
WindowID           1603111 non-null int64
Split              1603111 non-null object
ApplicationDate    1603111 non-null object
JobID              1603111 non-null int64
dtypes: int64(3), object(2)
memory usage: 61.2+ MB


In [0]:
app.describe()

In [0]:
app.head()

In [14]:
app[app.isnull().any(axis=1)]

,UserID,WindowID,Split,ApplicationDate,JobID


No missing data detected

In [0]:
app.groupby('UserID')['UserID'].nunique()

*Popular_job.csv*

In [0]:
popular_job = pd.read_csv('popular_jobs.csv')

In [12]:
popular_job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22838 entries, 0 to 22837
Data columns (total 2 columns):
UserId     22838 non-null int64
 JobIds    22762 non-null object
dtypes: int64(1), object(1)
memory usage: 356.9+ KB


In [0]:
popular_job.UserId.nunique()

*Jobs dataset*

In [11]:
! unzip jobs.zip

Archive:  jobs.zip
  inflating: jobs.tsv                


In [0]:
!ls

In [12]:
jobs = pd.read_csv('jobs.tsv',delimiter='\t',encoding='utf-8',error_bad_lines=False)

b'Skipping line 122433: expected 11 fields, saw 12\n'
b'Skipping line 602576: expected 11 fields, saw 12\n'
b'Skipping line 990950: expected 11 fields, saw 12\n'
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
jobs.info()

In [0]:
jobs.tail(10)

In [0]:
jobs['Description'].head(10)

**Building Recommender System**

Popular Job table

In [0]:
popular_job[' JobIds'].head(10)

In [25]:
popular_job.UserId.values

array([    767,     769,     861, ..., 1467586, 1470538, 1470809])

In [0]:
# Create dataframe for job applied
import re
user_ids = []
job_ids = []
loop_id = popular_job.UserId.values
for user_id in loop_id:
  applied_job = popular_job[' JobIds'].loc[popular_job['UserId'] == user_id].values.tolist()
  applied_job = str(applied_job)
  applied_job = re.sub('[^A-Za-z0-9_\s]+', '', applied_job)
  applied_job = applied_job.split()
  for job in applied_job:
    user_ids.append(user_id)
    job_ids.append(job)
jobs_tuple = list(zip(user_ids,job_ids))
df_applied = pd.DataFrame(jobs_tuple, columns=['Ids','JobID'])

In [14]:
df_applied.set_index('Ids')

,JobID
Ids,
767,299322
767,602736
767,1077691
767,625281
767,898514
767,1035141
767,659377
767,370120
767,370121


In [0]:
# Add binary label
df_applied['binary_label'] = 1

In [0]:
df_applied.head()

*Creat dataframe for job not applied*

In [0]:
df_jobs = pd.DataFrame()
df_jobs['JobID'] = jobs['JobID'].copy()

In [0]:
# Extracting jobid for test
#user_id: 767
selected_user_id = 767
df_jobs_test['Ids'] = selected_user_id

In [0]:
df_jobs_test.set_index('Ids')

In [0]:
df_applied_test = df_applied[df_applied['Ids'] == selected_user_id]
df_applied_test.head()

In [0]:
df_jobs_test['JobID'].dtype

In [0]:
df_applied_test['JobID'] = df_applied_test['JobID'].astype(int)

In [0]:
df_applied_test['JobID'].dtype

In [0]:
df_merge_test = pd.merge(df_jobs_test,df_applied_test,on=['JobID'],how='left')

In [0]:
df_merge_test = df_merge_test.drop('Ids_y',axis=1)

In [0]:
df_merge_test['binary_label'] = df_merge_test['binary_label'].fillna(0)

In [0]:
df_merge_test.set_index('Ids_x')

In [0]:
# Select the data for training (20%-1, 80%-0)
df_test_data_1 = df_merge_test[df_merge_test['binary_label'] == 1]
test_data_1_length = int(df_test_data_1.shape[0])

In [0]:
data_ratio = 0.2
test_data_0_length = int((test_data_1_length / data_ratio) - test_data_1_length)
print(test_data_0_length)

In [0]:
df_test_data_0 = df_merge_test[df_merge_test['binary_label'] == 0]
df_test_data_0 = df_test_data_0.sample(n=test_data_0_length,random_state=1234)

In [0]:
df_test_data_1.set_index('Ids_x')
df_test_data_0.set_index('Ids_x')
df_test_data_0.head(20)

In [0]:
df_test_data_1.head(2)

In [0]:
# concat for test
df_concat_test = pd.concat([df_test_data_1,df_test_data_0])

In [0]:
df_concat_test.tail(40)

In [23]:
!pip install numba

In [0]:
df_applied.drop(df_applied[df_applied.JobID == 'nan'].index,inplace=True)

In [0]:
df_applied['JobID'] = df_applied['JobID'].astype(int)

In [0]:
training_data.info()

***Combining df_applied and df_jobs for user_ids***

In [0]:
df_applied.drop(df_applied[df_applied.JobID == 'nan'].index,inplace=True)
df_applied['JobID'] = df_applied['JobID'].astype(int)

In [18]:
df_applied['JobID'].dtype

dtype('int64')

In [19]:
df_jobs['JobID'].dtype

dtype('int64')

In [0]:
df_jobs['binary_label'] = int(0)

In [29]:
df_jobs.head()

,JobID,binary_label
0,1,0
1,4,0
2,7,0
3,8,0
4,9,0


In [30]:
df_applied.index

Int64Index([      0,       1,       2,       3,       4,       5,       6,
                  7,       8,       9,
            ...
            3413814, 3413815, 3413816, 3413817, 3413818, 3413819, 3413820,
            3413821, 3413822, 3413823],
           dtype='int64', length=3413748)

In [0]:
# try combining two dataframe with left join
combined_test = pd.merge(df_jobs,df_applied,on='JobID',how='left')

In [22]:
combined_test[combined_test['Ids'].notnull()]

,JobID,binary_label_x,Ids,binary_label_y
2,7,0,1082218.0,1.0
6,11,0,4639.0,1.0
7,11,0,26275.0,1.0
8,11,0,36199.0,1.0
9,11,0,98819.0,1.0
10,11,0,101953.0,1.0
11,11,0,104975.0,1.0
12,11,0,106229.0,1.0
13,11,0,110599.0,1.0
14,11,0,173987.0,1.0


In [0]:
combined_test =  combined_test.drop('binary_label_x', axis=1)

In [24]:
combined_test.head()

,JobID,Ids,binary_label_y
0,1,NaN,NaN
1,4,NaN,NaN
2,7,1082218.0,1.0
3,8,NaN,NaN
4,9,NaN,NaN


In [0]:
combined_test['binary_label_y'].fillna(0,inplace=True)

In [43]:
combined_test[combined_test['Ids'].isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 745167 entries, 0 to 4158846
Data columns (total 3 columns):
JobID             745167 non-null int64
Ids               0 non-null float64
binary_label_y    745167 non-null float64
dtypes: float64(2), int64(1)
memory usage: 22.7 MB


In [44]:
combined_test[combined_test['Ids'].notnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3413680 entries, 2 to 4158845
Data columns (total 3 columns):
JobID             int64
Ids               float64
binary_label_y    float64
dtypes: float64(2), int64(1)
memory usage: 104.2 MB


In [0]:
# select random ID to fill in the null Ids
import random
np.random.seed(42)
id_list = df_applied.Ids.values.tolist()
combined_test['Ids'] =  combined_test['Ids'].fillna(pd.Series(np.random.choice(id_list,len(combined_test.index))))

In [50]:
combined_test.tail(20)

,JobID,Ids,binary_label_y
4158827,1116185,1116492.0,1.0
4158828,1116186,1234647.0,1.0
4158829,1116187,515227.0,0.0
4158830,1116188,352283.0,0.0
4158831,1116189,781105.0,0.0
4158832,1116190,264666.0,1.0
4158833,1116190,570891.0,1.0
4158834,1116191,1269024.0,0.0
4158835,1116196,1042805.0,0.0
4158836,1116197,933810.0,1.0


In [25]:
combined_test[['Ids','JobID','binary_label_y']]

,Ids,JobID,binary_label_y
0,903471.0,1,0.0
1,1122887.0,4,0.0
2,1082218.0,7,1.0
3,1291691.0,8,0.0
4,945420.0,9,0.0
5,234785.0,10,0.0
6,4639.0,11,1.0
7,26275.0,11,1.0
8,36199.0,11,1.0
9,98819.0,11,1.0


In [0]:
combined_test['Ids'] = combined_test['Ids'].astype(int)
combined_test['binary_label_y'] = combined_test['binary_label_y'].astype(int)
combined_test['JobID'] = combined_test['JobID'].astype(int)


# Binary Recommendation with PySpark

In [0]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import col

*Create Schema*

In [0]:
mySchema = StructType([StructField("Ids", IntegerType(), True)\
                      ,StructField("JobID", IntegerType(), True)\
                      ,StructField("binary_label_y", IntegerType(), True)])

*Create Spark dataframe*

In [0]:
df_spark = spark.createDataFrame(combined_test,schema=mySchema)

Test Dataframe

In [54]:
df_spark.show()

+---+-------+--------------+
|Ids|  JobID|binary_label_y|
+---+-------+--------------+
|  1| 903471|             0|
|  4|1122887|             0|
|  7|1082218|             1|
|  8|1291691|             0|
|  9| 945420|             0|
| 10| 234785|             0|
| 11|   4639|             1|
| 11|  26275|             1|
| 11|  36199|             1|
| 11|  98819|             1|
| 11| 101953|             1|
| 11| 104975|             1|
| 11| 106229|             1|
| 11| 110599|             1|
| 11| 173987|             1|
| 11| 181416|             1|
| 11| 192095|             1|
| 11| 205370|             1|
| 11| 218817|             1|
| 11| 236844|             1|
+---+-------+--------------+
only showing top 20 rows



Calculate Sparity

In [30]:
# Count total number of binary_label in the dataset
numerator = df_spark.select("binary_label_y").count()
# Count the number of distinct Ids and distinct JobID
num_users = df_spark.select("Ids").distinct().count()
num_jobs = df_spark.select("JobID").distinct().count()
# Set the denominator equal to the number of users multiplied by the number of jobs
denominator = num_users * num_jobs
# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print ("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  99.98% empty.


Create test/train splits and create ALS Model

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [0]:
# Create test and train set
(train, test) = df_spark.randomSplit([0.8, 0.2], seed = 1234)

In [0]:
# Empty list to be filled with model
model_list = []
# Complete parameters
ranks = [10,20,30,40]
maxIters = [10,20,30,40]
regParams = [.05,.1,.15]
alphas = [20,40,60,80]

In [0]:
# # For loop will automatically create and store ALS models
# for r in ranks:
#     for mi in maxIters:
#         for rp in regParams:
#             for a in alphas:
#                 model_list.append(ALS(userCol= "userId", itemCol= "songId", ratingCol= "num_plays", rank = r, maxIter = mi, regParam = rp, alpha = a, coldStartStrategy="drop", nonnegative = True, implicitPrefs = True))

# # Print the model list, and the length of model_list
# print (model_list, "Length of model_list: ", len(model_list))

# # Validate
# len(model_list) == (len(ranks)*len(maxIters)*len(regParams)*len(alphas))

In [0]:
# Create ALS model
als = ALS(userCol="Ids", itemCol="JobID", ratingCol="binary_label_y",rank = 10, maxIter = 10, regParam = .05, alpha = 20, coldStartStrategy="drop", nonnegative = True, implicitPrefs = False)

In [37]:
# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

In [0]:
# ROEM: rank ordering error metric
def ROEM(predictions, userCol = "Ids", itemCol = "JobID", ratingCol = "binary_label_y"):
  #Creates table that can be queried
  predictions.createOrReplaceTempView("predictions")

  #Sum of total number of plays of all songs
  denominator = predictions.groupBy().sum(ratingCol).collect()[0][0]

  #Calculating rankings of songs predictions by user
  spark.sql("SELECT " + userCol + " , " + ratingCol + " , PERCENT_RANK() OVER (PARTITION BY " + userCol + " ORDER BY prediction DESC) AS rank FROM predictions").createOrReplaceTempView("rankings")

  #Multiplies the rank of each song by the number of plays and adds the products together
  numerator = spark.sql('SELECT SUM(' + ratingCol + ' * rank) FROM rankings').collect()[0][0]

  performance = numerator/denominator

  return performance


In [0]:
# fitting model
model = als.fit(train)
# Generating prediction
predictions = model.transform(test)

In [0]:
#Calculating expected percentile rank error metric for the model on each fold's prediction set
performance = ROEM(predictions)

In [42]:
print("Test Percent Rank Errors: ")
print(performance)

Test Percent Rank Errors: 
0.4511608309275242


In [0]:
df_pred = predictions.toPandas()

In [57]:
df_pred['binary_label_y'].value_counts()

1    651734
Name: binary_label_y, dtype: int64